In [29]:
%matplotlib inline

import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
import pickle, os, gc, re, sys
from datetime import datetime 

In [6]:
# import pyreadstat
# did not help!

In [2]:
BES_Panel = pd.read_pickle("..\\BES_analysis_data\\"+"W19_comb"+os.sep+"BES2019_W19_Panel_v0.1.zip",compression='zip')

In [1]:
# %%time
# cat_vars = BES_Panel.columns[ BES_Panel.dtypes=="category" ]
# BES_Panel[cat_vars].apply(lambda x: x.cat.set_categories([str(x) for x in x.cat.categories]), axis=0, result_type='reduce')

In [20]:
%%time
# make sure all categories in category variables are strings (mixed dtypes caused issues)
BES_Panel[cat_vars] = BES_Panel[cat_vars].apply(lambda x: x.cat.set_categories([str(x) for x in x.cat.categories]), axis=0, result_type='reduce')

Wall time: 12.4 s


In [2]:
# BES_Panel = pd.read_stata('../BES_analysis_data/'+"W19_comb"+os.sep+"BES2019_W19_Panel_v0.1.dta")
# crashes with a memory error

In [12]:
def load_large_dta(fname, chunksize=20000, columns=None):
    
    counter=0
    reader = pd.read_stata(fname, chunksize=chunksize, columns=columns)
    df = pd.DataFrame()
    print("started")
    for chunk in reader:
        df = df.append(chunk)

        print(counter,datetime.now())
        sys.stdout.flush()
        gc.collect()
        counter=counter+1

    print('\nloaded {} rows'.format(len(df)))

    return df

In [62]:
BES_Panel4 = load_large_dta("..\\BES_analysis_data\\"+"W19_comb"+os.sep+"BES2019_W19_Panel_v0.1.dta", chunksize=20000)
# note, this crashed when I ran it with *too small a chunk size* (1000)
# bingo!

started
0 2020-03-07 04:36:50.149743
1 2020-03-07 04:38:44.813302
2 2020-03-07 04:40:56.675844
3 2020-03-07 04:44:19.885467
4 2020-03-07 04:48:23.774417

loaded 97158 rows


In [64]:
BES_Panel4.dtypes.value_counts()

# no bueno!
# object      1360

category    1774
object      1360
category     403
category     310
category     199
            ... 
category       1
category       1
category       1
category       1
category       1
Length: 413, dtype: int64

In [9]:
# # reload just the variables mis-loaded as "object" type
# gc.collect()

BES_Panel5 = pd.read_stata("..\\BES_analysis_data\\"+"W19_comb"+os.sep+"BES2019_W19_Panel_v0.1.dta",
                        columns=list( BES_Panel4.columns[ BES_Panel4.dtypes=="object" ] ))


In [74]:

gc.collect()
BES_Panel5.dtypes.value_counts()

category    713
category     47
category     32
category     30
category     28
           ... 
category      1
category      1
category      1
category      1
category      1
Length: 257, dtype: int64

In [80]:
# double-check - does it load the categories correctly now? Ys it does
BES_Panel5[BES_Panel4.columns[ BES_Panel4.dtypes=="object" ][5]]


0                       NaN
1                       NaN
2                       NaN
3        Completely certain
4                       NaN
                ...        
97153    Completely certain
97154                     6
97155                   NaN
97156                   NaN
97157                   NaN
Name: generalElectionCertaintyUnsqW5, Length: 97158, dtype: category
Categories (8, object): [Not at all certain < 2 < 3 < 4 < 5 < 6 < Completely certain < Don't know]

In [225]:
# So, basically fixed, right?

In [119]:
BES_Panel6=BES_Panel4.copy()
BES_Panel6[BES_Panel5.columns] = BES_Panel5
BES_Panel6.dtypes[BES_Panel6.dtypes!='category'].value_counts()

float64           157
datetime64[ns]     43
float32            22
object              7
int32               1
dtype: int64

In [3]:
# Not quite - still 7 objects

In [92]:
BES_Panel6[ BES_Panel6.columns[BES_Panel6.dtypes=='object'] ]

,xtoday_dateW18,xyesterday_dateW18,xbefore_yesterday_dateW18,xbefore_yesterday_dayW18,uniname_statusW19,JACS_subjectW19,euRefLA
0,20191126,20191125,20191124,Sunday,__NA__,,W06000012
1,,,,,,H,W06000012
2,,,,,,,
3,20191126,20191125,20191124,Sunday,__NA__,,W06000012
4,20191130,20191129,20191128,Thursday,__NA__,,
...,...,...,...,...,...,...,...
97153,,,,,,,
97154,,,,,,,E07000211
97155,,,,,,,E09000003
97156,,,,,,,


In [93]:
BES_Panel7 = pd.read_stata("..\\BES_analysis_data\\"+"W19_comb"+os.sep+"BES2019_W19_Panel_v0.1.dta",
                        columns=list( BES_Panel6.columns[ BES_Panel6.dtypes=="object" ] ))
 # - and not due to loading errors into python

In [115]:
# pd.to_datetime(
BES_Panel7["xbefore_yesterday_dayW18"].value_counts()
               #.replace("",np.nan),format="%Y%m%d").value_counts()

             59333
Friday        5789
Saturday      5562
Monday        5434
Tuesday       5420
Thursday      5295
Sunday        5196
Wednesday     5129
Name: xbefore_yesterday_dayW18, dtype: int64

In [109]:
BES_Panel7["xtoday_dateW18"].value_counts()

            59333
20191208     1530
20191201     1467
20191202     1434
20191209     1429
20191211     1428
20191114     1420
20191126     1415
20191124     1412
20191117     1380
20191127     1375
20191119     1373
20191130     1365
20191205     1352
20191125     1351
20191118     1348
20191115     1347
20191207     1343
20191128     1332
20191120     1318
20191203     1316
20191121     1314
20191204     1310
20191123     1305
20191116     1282
20191129     1279
20191206     1263
20191122     1240
20191210     1092
20191113        3
20191212        2
Name: xtoday_dateW18, dtype: int64

In [224]:
BES_Panel6["JACS_subjectW19"]

0        NaN
1          H
2        NaN
3        NaN
4        NaN
        ... 
97153    NaN
97154    NaN
97155    NaN
97156    NaN
97157    NaN
Name: JACS_subjectW19, Length: 97158, dtype: category
Categories (21, object): [A, B, C, D, ..., V, W, X, Y]

In [213]:
BES_Panel7["JACS_subjectW19"] = BES_Panel7["JACS_subjectW19"].replace("",np.nan).astype('category')
BES_Panel7["uniname_statusW19"] = BES_Panel7["uniname_statusW19"].replace("",np.nan).astype('category')
BES_Panel7["euRefLA"] = BES_Panel7["euRefLA"].replace("",np.nan).astype('category')
BES_Panel7["xtoday_dateW18"]     = pd.to_datetime(BES_Panel7["xtoday_dateW18"].replace("",np.nan),format="%Y%m%d")
BES_Panel7["xyesterday_dateW18"] = pd.to_datetime(BES_Panel7["xyesterday_dateW18"].replace("",np.nan),format="%Y%m%d")
BES_Panel7["xbefore_yesterday_dateW18"] = pd.to_datetime(BES_Panel7["xbefore_yesterday_dateW18"].replace("",np.nan),format="%Y%m%d")
BES_Panel7["xbefore_yesterday_dayW18"] = BES_Panel7["xbefore_yesterday_dayW18"].replace("",np.nan).astype('category')

BES_Panel6[BES_Panel7.columns] = BES_Panel7
# types fixed

In [ ]:
# what about the non-object types? Are they all kosher?

# Not entirely

# couple of straight-up bugs (in the original dta or due to an invisible loading error?)
# some issues where things are floats when they should be bools
# some issues where things are floats when they should be categories
# some issues where things are floats because they have nans but - using the new panda dtypes - can actually be represented as their proper dtype with nans having their own special <NA> value

In [11]:
# "wt_(new|full|daily)" - all legit

# wave\d+$ -> boolean

# fatherVoteW14W15W16/motherVoteW14W15W16 -> manual fix -> category
# countryW\d+$ -> manual fix -> category



# discussantturnoutName -> category
# "warm.*W7" -> category (0-100pt +DK)
# partyAgree2W12 -> category (0-100pt +DK)
# generalElecCertainty2squeezeW17 -> category (1-7 +DK)
# "jobzone" -> category (could maybe be "sector" variable?)

# category_var_str = "(discussantturnoutName|warm.*W7$|partyAgree2W12|generalElecCertainty2squeezeW17|jobzone)"
# BES_Panel6[ match(BES_Panel6,category_var_str).index ] = BES_Panel6[ match(BES_Panel6,category_var_str).index ].astype('category')

# "w\d+full" -> boolean with nans

# "pano" -> 1-650 (uint16)
# mapNames -> ints -> you should change this to INTS with nans

# "CampaignDayW\d+" -> uint8 with nans
# riskScaleW8 -> uint8 with nans
# identityStrengthAllW11 -> uint8 with nans
# ptvexpgrpW14 -> ??? 1-7 unit8 with nans
# Age: 15 -> 116 uint8 with nans
# "ageW\d+$" -> uint8 with nans

# numeric_var_str = "(Age$|ageW\d+$|ptvexpgrpW14|identityStrengthAllW11|riskScaleW8|CampaignDayW\d+$|mapNames|pano|w\d+full$)"
# BES_Panel6[ match(BES_Panel6,numeric_var_str).index ] = BES_Panel6[ match(BES_Panel6,numeric_var_str).index ].astype('category')

In [214]:
BES_Panel6[search(BES_Panel6,"wave\d+$").index] = BES_Panel6[search(BES_Panel6,"wave\d+$").index].astype('bool')

motherVote_dict = {0:"Did not vote",1:"Labour",2:"Conservatives",3:"Liberal Democrats/Liberals/SDP/Alliance",
                   4:"Scottish National Party (SNP)", 5:"Plaid Cymru",9:"Other",10:"Had no usual party",
                   9999:"Don't know",98:"Did not vote"} # "Didn't know mother growing up"???
fatherVote_dict = {0:"Did not vote",1:"Labour",2:"Conservatives",3:"Liberal Democrats/Liberals/SDP/Alliance",
                   4:"Scottish National Party (SNP)", 5:"Plaid Cymru",9:"Other",10:"Had no usual party",
                   9999:"Don't know",98:"Did not vote"} # "Didn't know father growing up"???

# some guesswork in this one!

BES_Panel6["fatherVoteW14W15W16"] = BES_Panel6["fatherVoteW14W15W16"].replace(fatherVote_dict)
BES_Panel6["motherVoteW14W15W16"] = BES_Panel6["motherVoteW14W15W16"].replace(motherVote_dict)

country_dict={1:"English",2:"Scotland",3:"Wales"}
BES_Panel6[ match(BES_Panel6,"countryW\d+$").index] = BES_Panel6[match(BES_Panel6,"countryW\d+$").index].replace(country_dict)

category_var_str = "(discussantturnoutName|warm.*W7$|partyAgree2W12|generalElecCertainty2squeezeW17|jobzone)"
BES_Panel6[ match(BES_Panel6,category_var_str).index ] = BES_Panel6[ match(BES_Panel6,category_var_str).index ].astype('category')

numeric_var_str = "(Age$|ageW\d+$|ptvexpgrpW14|identityStrengthAllW11|riskScaleW8|CampaignDayW\d+$|mapNames|pano|w\d+full$)"
BES_Panel6[ match(BES_Panel6,numeric_var_str).index ] = BES_Panel6[ match(BES_Panel6,numeric_var_str).index ].astype('float32')

In [218]:
BES_Panel6["fatherVoteW14W15W16"] = BES_Panel6["fatherVoteW14W15W16"].astype('category')
BES_Panel6["motherVoteW14W15W16"] = BES_Panel6["motherVoteW14W15W16"].astype('category')
BES_Panel6[ match(BES_Panel6,"countryW\d+$").index] = BES_Panel6[match(BES_Panel6,"countryW\d+$").index].astype('category')


In [220]:
numeric_var_str = "(Age$|ageW\d+$|ptvexpgrpW14|identityStrengthAllW11|riskScaleW8|CampaignDayW\d+$|mapNames|pano|w\d+full$)"
BES_Panel6[ match(BES_Panel6,numeric_var_str).index ] = BES_Panel6[ match(BES_Panel6,numeric_var_str).index ].astype('float32')

In [221]:
BES_Panel6.dtypes[BES_Panel6.dtypes!='category'].value_counts()

float64           69
float32           51
datetime64[ns]    46
bool              19
int32              1
dtype: int64

In [ ]:
BES_Panel.to_pickle("..\\BES_analysis_data\\"+"W19_comb"+os.sep+"BES2019_W19_Panel_v0.1.zip",compression='zip')

In [ ]:
#### well-deserved break

In [ ]:
# load and downcast/convert to nannable types

In [ ]:
BES_Panel = pd.read_pickle("..\\BES_analysis_data\\"+"W19_comb"+os.sep+"BES2019_W19_Panel_v0.1.zip",compression='zip')

In [ ]:
BES_Panel.dtypes[BES_Panel.dtypes!='category'].value_counts()

In [ ]:
float64_vars = BES_Panel.columns[BES_Panel.dtypes=='float64'] # all weights
large_int_vars = match(BES_Panel,"pano|mapNames").index
small_int_vars = match(BES_Panel,"Age$|ageW\d+$|ptvexpgrpW14|identityStrengthAllW11|riskScaleW8|CampaignDayW\d+$").index
boolean_vars = match(BES_Panel,"w\d+full").index

# cogempathyIRTW14	affempathyIRTW14	zeroSumIRTW14 -> really are floats

## note - all downcasting is "safe" because it throws an error if you try to cast in such a way that
## your reducing a variable below the maximum size it needs to represent it's value

In [ ]:
# downcast to ordinary pandas dtypes
BES_Panel[float64_vars] = BES_Panel[float64_vars] = BES_Panel[float64_vars].apply( lambda x: pd.to_numeric(x, downcast='float') )
# BES_Panel["id"] = BES_Panel["id"].apply( lambda x: pd.to_numeric(x, downcast='unsigned') ) # technically uint32 is sufficient, but I'll leave this one be

In [ ]:
BES_Panel.to_stata("..\\BES_analysis_data\\"+"W19_comb"+os.sep+"BES2019_W19_Panel_v0.1_cleaned.dta")
# to_stata can't handle nannable types

# default is most widely accessible Version 114
# but could also use Version 117 in which case strL variables names can be used -> output dta is then smaller

In [ ]:
# downcast to new Nannnable dtypes
BES_Panel[large_int_vars] = BES_Panel[large_int_vars].astype("Int16")
BES_Panel[boolean_vars] = BES_Panel[boolean_vars] = BES_Panel[boolean_vars].astype('boolean')
BES_Panel[small_int_vars] = BES_Panel[small_int_vars].replace(9999,np.nan).astype("Int8")


In [ ]:
BES_Panel.to_pickle("..\\BES_analysis_data\\"+"W19_comb"+os.sep+"BES2019_W19_Panel_v0.1_cleaned.zip",compression='zip')
# zipped pickle works fine as storage/quick load option

In [ ]:
# careful check that the zipped pickle stores everything okay

In [25]:
BES_Panel_check = pd.read_pickle("..\\BES_analysis_data\\"+"W19_comb"+os.sep+"BES2019_W19_Panel_v0.1_cleaned.zip",compression='zip')
# zipped pickle works fine as storage/quick load option

In [26]:
BES_Panel_check.shape

(97158, 7911)

In [27]:
BES_Panel_check.dtypes.value_counts()

category    2487
category     403
category     331
category     205
category     173
            ... 
category       1
category       1
category       1
category       1
category       1
Length: 579, dtype: int64

In [28]:
BES_Panel_check.dtypes[BES_Panel_paranoia.dtypes!='category'].value_counts()

category          475
float32            72
datetime64[ns]     46
category           31
category           28
                 ... 
category            1
category            1
category            1
category            1
category            1
Length: 249, dtype: int64

In [ ]:
# perfect!

In [ ]:
# paranoid check on the "cleaned" dta output (it's half the size of the original!)

In [ ]:
# BES_Panel = pd.read_stata("../BES_analysis_data/"+"W19_comb"+os.sep+"BES2019_W19_Panel_v0.1_cleaned.dta")
# even cleaned I can't load this direct (on a 16Gb machine)

In [39]:
%%time
BES_Panel_paranoia = load_large_dta("..\\BES_analysis_data\\"+"W19_comb"+os.sep+"BES2019_W19_Panel_v0.1_cleaned.dta", chunksize=10)
# note, this crashed when I ran it with *too small a chunk size* (1000)
# bingo!

started
0 2020-03-10 18:02:22.455099
1 2020-03-10 18:03:06.285605
2 2020-03-10 18:03:51.544194
3 2020-03-10 18:04:38.139859
4 2020-03-10 18:05:20.672292
5 2020-03-10 18:06:02.682695
6 2020-03-10 18:06:43.392023
7 2020-03-10 18:07:24.435371
8 2020-03-10 18:08:06.812795
9 2020-03-10 18:08:50.032267
10 2020-03-10 18:09:34.099787
11 2020-03-10 18:10:18.404321
12 2020-03-10 18:11:04.000929
13 2020-03-10 18:11:50.012561
14 2020-03-10 18:12:36.584225
15 2020-03-10 18:13:24.070941
16 2020-03-10 18:14:12.450708
17 2020-03-10 18:15:01.583518
18 2020-03-10 18:15:51.020346
19 2020-03-10 18:16:41.435229
20 2020-03-10 18:17:32.591155
21 2020-03-10 18:18:24.279112
22 2020-03-10 18:19:17.159136
23 2020-03-10 18:20:10.492187
24 2020-03-10 18:21:03.801236
25 2020-03-10 18:22:01.208519
26 2020-03-10 18:23:00.326901
27 2020-03-10 18:23:59.570289
28 2020-03-10 18:25:02.782905
29 2020-03-10 18:26:12.643901
30 2020-03-10 18:27:16.746567
31 2020-03-10 18:28:15.860948
32 2020-03-10 18:29:23.704829
33 2020-03-1

KeyboardInterrupt: 

In [20]:
BES_Panel_paranoia.shape

(97158, 7912)

In [21]:
BES_Panel_paranoia.dtypes.value_counts()

category    2012
object       999
category     403
category     310
category     199
            ... 
category       1
category       1
category       1
category       1
category       1
Length: 435, dtype: int64

In [22]:
BES_Panel_paranoia.dtypes[BES_Panel_paranoia.dtypes!='category'].value_counts()

object            999
float32           120
datetime64[ns]     46
int8               19
int32               2
dtype: int64

In [24]:
# objects still an issue with chunk iteration on slightly slimmed down dta

In [30]:
BES_Panel = pd.read_pickle("..\\BES_analysis_data\\"+"W19_comb"+os.sep+"BES2019_W19_Panel_v0.1_cleaned.zip",compression='zip')
# zipped pickle works fine as storage/quick load option

In [33]:
from utility import *
search(BES_Panel_check, "motherVote")

motherVoteW17           3564
motherVoteW14W15W16    49947
dtype: int64

In [37]:
BES_Panel_check["motherVoteW17"].value_counts()

Labour                                     1239
Don't know                                  929
Conservatives                               744
Liberal Democrats/Liberals/SDP/Alliance     217
Did not vote                                170
Had no usual party                          104
Didn't know mother growing up                51
Other                                        50
Scottish National Party (SNP)                29
UKIP                                         21
Plaid Cymru                                  10
Name: motherVoteW17, dtype: int64

In [36]:
BES_Panel_check["motherVoteW14W15W16"].value_counts()

Labour                                     16286
Don't know                                 13182
Conservatives                              12424
Liberal Democrats/Liberals/SDP/Alliance     3006
Didn't know father growing up               1952
Had no usual party                          1248
Did not vote                                 815
Other                                        481
Scottish National Party (SNP)                379
Plaid Cymru                                  174
Name: motherVoteW14W15W16, dtype: int64

In [38]:
BES_Panel_check["motherVoteW17"].cat.categories

Index(['Did not vote', 'Labour', 'Conservatives',
       'Liberal Democrats/Liberals/SDP/Alliance',
       'Scottish National Party (SNP)', 'Plaid Cymru', 'UKIP', 'Other',
       'Had no usual party', 'Didn't know mother growing up', 'Don't know'],
      dtype='object')